In [ ]:
import numpy as np
import os
import subprocess
import shutil
import open3d

In [ ]:
train_set = [
    "sg27_station4_intensity_rgb",
    "sg27_station5_intensity_rgb",
    "sg27_station9_intensity_rgb",
    "sg28_station4_intensity_rgb",
    "untermaederbrunnen_station1_xyz_intensity_rgb",
    "untermaederbrunnen_station3_xyz_intensity_rgb",
]
valid_set = [
    "bildstein_station1_xyz_intensity_rgb",
    "bildstein_station3_xyz_intensity_rgb",
    "bildstein_station5_xyz_intensity_rgb",
    "domfountain_station1_xyz_intensity_rgb",
    "domfountain_station2_xyz_intensity_rgb",
    "domfountain_station3_xyz_intensity_rgb",
    "neugasse_station1_xyz_intensity_rgb",
    "sg27_station1_intensity_rgb",
    "sg27_station2_intensity_rgb",
]
test_set = [
    "birdfountain_station1_xyz_intensity_rgb",
    "castleblatten_station1_intensity_rgb",
    "castleblatten_station5_xyz_intensity_rgb",
    "marketplacefeldkirch_station1_intensity_rgb",
    "marketplacefeldkirch_station4_intensity_rgb",
    "marketplacefeldkirch_station7_intensity_rgb",
    "sg27_station10_intensity_rgb",
    "sg27_station3_intensity_rgb",
    "sg27_station6_intensity_rgb",
    "sg27_station8_intensity_rgb",
    "sg28_station2_intensity_rgb",
    "sg28_station5_xyz_intensity_rgb",
    "stgallencathedral_station1_intensity_rgb",
    "stgallencathedral_station3_intensity_rgb",
    "stgallencathedral_station6_intensity_rgb",
]
all_set = train_set + valid_set + test_set

In [ ]:
def wc(file_name):
    out = subprocess.Popen(['wc', '-l', file_name],
                           stdout=subprocess.PIPE,
                           stderr=subprocess.STDOUT
                          ).communicate()[0]
    return int(out.partition(b' ')[0])

def prepend_line(file_name, line):
    with open(file_name, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(line.rstrip('\r\n') + '\n' + content)
        
def point_cloud_txt_to_pcd(raw_data_dir, file_prefix):
    # File names
    txt_file = os.path.join(raw_data_dir, file_prefix + '.txt')
    pts_file = os.path.join(raw_data_dir, file_prefix + '.pts')
    pcd_file = os.path.join(raw_data_dir, file_prefix + '.pcd')
    
    # Skip if already done
    if os.path.isfile(pcd_file):
        print("pcd {} exists, skipped".format(pcd_file))
        return

    # .txt to .pts
    print("[txt->pts]")
    print("txt: {}".format(txt_file))
    print("pts: {}".format(pts_file))
    shutil.copyfile(txt_file, pts_file)
    prepend_line(pts_file, str(wc(txt_file)))

    # .pts -> .pcd
    print("[pts->pcd]")
    print("pts: {}".format(pts_file))
    print("pcd: {}".format(pcd_file))
    point_cloud = open3d.read_point_cloud(pts_file)
    open3d.write_point_cloud(pcd_file, point_cloud)
    os.remove(pts_file)

    # Sanity check
    point_cloud_loaded = open3d.read_point_cloud(pcd_file)
    np.testing.assert_allclose(np.asarray(point_cloud.points),
                               np.asarray(point_cloud_loaded.points))

In [ ]:
raw_data_dir = "/home/ylao/data/semantic3d"

for file_prefix in all_set:
    point_cloud_txt_to_pcd(raw_data_dir, file_prefix)